In [1]:
import pandas as pd
import numpy as np
import pyodbc as db
import warnings
from numpy.random import seed
from numpy.random import randn
from numpy.random import normal
from scipy.stats import ttest_ind
warnings.filterwarnings('ignore')

In [3]:
cnx: db.Connection = db.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")

In [4]:
users = pd.read_excel('Z:\\Analytics\\High Values Change Analysis\\High Users.xlsx')
users['Given High Status-Date'] = pd.to_datetime(users['Given High Status-Date'], format='%Y-%m-%d')

In [5]:
def get_deposit_history(TOTOID, UsersList):
    user_bets = f"""
    DECLARE @StartDate DATE;
    DECLARE @EndDate   DATE;
    SET @StartDate = '2022-07-01'
    SET @EndDate   = CAST(GETDATE() AS DATE)

    SELECT u.PartnerUserId,
    CAST(p.modify_date AS DATE)        AS Date,
    p.Amount                           AS Amount
    FROM Payment AS p
        INNER JOIN  C_PaymentSystem AS ps
            ON ps.PaymentSystemId = p.PaymentSystemID
        INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm AS u
            ON u.UserID = p.UserID
    WHERE u.PartnerUserId = {TOTOID}
      AND p.PaymentTypeID = 2
      AND p.PaymentStatusID = 8
      AND CAST(p.modify_date AS DATE)  < @EndDate
      AND CAST(p.modify_date AS DATE) >= @StartDate
      AND ps.PaymentSystemName NOT IN ('PokerTransfer','TRANSFER')
      AND u.PartnerID IN (237)
      AND p.SourceID = 2


    """
    df = pd.read_sql(user_bets, cnx)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['PartnerUserId'] = df['PartnerUserId'].astype(int)
    anchor_date = UsersList[UsersList['PartnerUserId'] == TOTOID].iloc[0,0]
    deposits_after = df[df['Date'] >= anchor_date]['Amount'].values
    deposits_before = df[df['Date'] < anchor_date]['Amount'].values
    t_stat, p_value = ttest_ind(deposits_before, deposits_after, equal_var=True)
    return t_stat, p_value

In [6]:
t_stats_dep = []
p_values_dep = []
for i in users['PartnerUserId'].values:
    t_stat, p_value = get_deposit_history(i, users)
    p_value = "{:.8f}".format(p_value)
    t_stat  = "{:.8f}".format(t_stat)
    print('Loop')
    print(t_stat, p_value)
    t_stats_dep.append(t_stat)
    p_values_dep.append(p_value)

Loop
3.91693146 0.00010338
Loop
6.79642337 0.00000000
Loop
-3.82009494 0.00015523
Loop
8.53284748 0.00000000
Loop
-3.18501189 0.00150971
Loop
20.27948253 0.00000000
Loop
3.17991402 0.00153360
Loop
2.07830264 0.03877646
Loop
-1.70033948 0.08953606
Loop
3.21171589 0.00148560
Loop
3.57090358 0.00036595
Loop
-4.95800768 0.00000083
Loop
-7.56920961 0.00000000
Loop
-0.41718914 0.67669030
Loop
-6.03742887 0.00000000
Loop
8.02605898 0.00000000
Loop
3.23356709 0.00130766
Loop
7.80648115 0.00000000
Loop
-8.99999051 0.00000000
Loop
2.86121704 0.00531076
Loop
-1.30848159 0.19143420
Loop
2.52419170 0.01185120
Loop
8.70349243 0.00000000
Loop
8.87908631 0.00000000
Loop
8.26525543 0.00000000
Loop
-0.74820217 0.45466215
Loop
4.48067927 0.00000846
Loop
4.52832781 0.00000864
Loop
-4.53046674 0.00000651
Loop
0.66847841 0.50400932
Loop
8.65470801 0.00000000
Loop
1.24690222 0.21429034
Loop
-0.85834302 0.39179608
Loop
nan nan
Loop
nan nan
Loop
-6.25785772 0.00000000
Loop
2.43892585 0.01518318
Loop
1.87802428

In [7]:
users['T_deposit'] = t_stats_dep
users['P_deposit'] = p_values_dep

In [9]:
users.to_excel('Deposit_t_test.xlsx')